## Merge Cluster Numbers with Original Data


In [1]:
import pandas as pd

In [2]:
CLUST_NUMS = "/shared/3/projects/newsDiffusion/data/interim/NEREmbedding/embeddingClusterList2020.pkl"
DF_PATH = "/shared/3/projects/newsDiffusion/data/processed/newsData/fullDataWithNER.tsv"
OUT_PATH = "/shared/3/projects/newsDiffusion/data/processed/fullDataWithClustNums.tsv"

In [3]:
clustDf = pd.read_pickle(CLUST_NUMS)

clustDf.head(2)

,clustNum,key
0,0,2580480
0,0,2121734


In [4]:
#load in main data source 
#we don't want to use "content", because it takes up a lot of space and
#we have already embedded the content. Can always merge back in later so long as we 
#keep the "key" column
LOAD_COLS = list(pd.read_csv(DF_PATH, \
                     nrows = 1, sep="\t").columns)
#LOAD_COLS.remove("content")
LOAD_COLS.remove("NamedEntities")

df = pd.read_csv(DF_PATH,\
                 sep="\t", usecols=LOAD_COLS)

/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (2,3,10,11,12,13,14,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df["date"] = pd.to_datetime(df["date"])

df["year"] = df["date"].dt.year

#filter so we only use 2020 where we have Reuters 
df = df[df["year"] == 2020]

In [6]:
#only keep rows for which we have a cluster number associated with the key 
df = pd.merge(df, clustDf, on="key", how="left")

In [7]:
#should be the same as length of clustDf or we have a merging issue
#luckily, no issue here 
print(len(df))

print(len(df.dropna(subset=["clustNum"])))

2442280
1266095


In [8]:
df = df.drop(columns=["Unnamed: 0"])
df.to_csv(OUT_PATH, sep="\t")